In [16]:
import io
import requests
import pandas as pd

from bokeh.models import ColumnDataSource, HoverTool, SaveTool
from bokeh.models.widgets import TextInput, Button
from bokeh.plotting import figure, curdoc, show
from bokeh.layouts import row, widgetbox
from bokeh.io import output_notebook

from ipywidgets import interact

TICKER = ""
base = "https://api.iextrading.com/1.0/"
data = ColumnDataSource(dict(time=[], display_time=[], price=[]))

output_notebook()
def get_last_price(symbol):
    payload = {
        "format": "csv",
        "symbols": symbol
    }
    endpoint = "tops/last"

    raw = requests.get(base + endpoint, params=payload)
    raw = io.BytesIO(raw.content)
    prices_df = pd.read_csv(raw, sep=",")
    prices_df["time"] = pd.to_datetime(prices_df["time"], unit="ms")
    prices_df["display_time"] = prices_df["time"].dt.strftime("%m-%d-%Y %H:%M:%S.%f")

    return prices_df

def update_ticker():
    global TICKER
    #TICKER = ticker_textbox.value
    if TICKER != 'Ticker':
        price_plot.title.text = "IEX Real-Time Price: " + ticker_textbox.value
        data.data = dict(time=[], display_time=[], price=[])

    return

def update_price():
    new_price = get_last_price(symbol=TICKER)
    data.stream(dict(time=new_price["time"],
                     display_time=new_price["display_time"],
                     price=new_price["price"]), 10000)
    return

def ticker_cb(t):
    global TICKER
    TICKER = t
    return TICKER
    

hover = HoverTool(tooltips=[
    ("Time", "@display_time"),
    ("IEX Real-Time Price", "@price")
    ])

price_plot = figure(plot_width=800,
                    plot_height=400,
                    x_axis_type='datetime',
                    tools=[hover, SaveTool()],
                    title="Real-Time Price Plot from IEX")


price_plot.line(source=data, x='time', y='price')
price_plot.xaxis.axis_label = "Time"
price_plot.yaxis.axis_label = "IEX Real-Time Price"
price_plot.title.text = "IEX Real Time Price: " + TICKER

#ticker_textbox = TextInput(placeholder="Ticker")
interact(ticker_cb, t="Ticker")
update = Button(label="Update")
interact()
update.on_click(update_ticker)

#inputs = widgetbox([ticker_textbox, update], width=200)

plotbox = row(inputs, price_plot, width=1600)
#curdoc().add_periodic_callback(update_price, 10000)
show(plotbox)

Loading BokehJS ...

interactive(children=(Text(value='Ticker', description='t'), Output()), _dom_classes=('widget-interact',))

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html

